In [1]:
import xarray as xr
import gsw_xarray as gsw


datadir = "../data/"

In [ ]:
ds_list = []
for i in range(1, 13):
    ds = xr.open_dataset(datadir + "originaldata/mimoc/" + f"MIMOC_Z_GRID_v2.2_CT_SA_month" + str(i).zfill(2) + ".nc")
    ds = ds.rename_dims(LONG="lon", LAT="lat", PRES="pressure")
    ds = ds.assign_coords(coords={"lon" : ds.LONGITUDE, "lat" : ds.LATITUDE, "pressure" : ds.PRESSURE})
    ds = ds.drop_vars(["LATITUDE", "LONGITUDE", "PRESSURE"])
    ds = ds.rename({"ABSOLUTE_SALINITY" : "sa", "CONSERVATIVE_TEMPERATURE" : "ct"})
    ds.coords["lon"] = (ds.coords["lon"] + 180) % 360 - 180
    ds = ds.sortby(ds.lon)
    ds = ds.sel(lat=slice(60, 90))
    ds = ds.expand_dims({"month" : [i]})
    ds_list.append(ds)
ds = xr.concat(ds_list, dim="month")
ds = ds.mean(dim="month", skipna=True)
ds

<xarray.Dataset> Size: 28MB
Dimensions:   (pressure: 81, lat: 61, lon: 720)
Coordinates:
  * lat       (lat) float32 244B 60.0 60.5 61.0 61.5 ... 88.5 89.0 89.5 90.0
  * pressure  (pressure) float32 324B 0.0 5.0 10.0 ... 1.85e+03 1.9e+03 1.95e+03
  * lon       (lon) float32 3kB -180.0 -179.5 -179.0 ... 178.5 179.0 179.5
Data variables:
    sa        (pressure, lat, lon) float32 14MB 33.07 33.02 32.9 ... 35.1 35.1
    ct        (pressure, lat, lon) float32 14MB 4.724 4.796 ... -0.6477 -0.6466

In [3]:
density = gsw.rho(SA=ds.sa, CT=ds.ct, p=ds.pressure)
density

<xarray.DataArray 'rho' (pressure: 81, lat: 61, lon: 720)> Size: 28MB
array([[[1026.05231697, 1026.00796828, 1025.91707901, ...,
         1026.1228215 , 1026.10145541, 1026.0766898 ],
        [1025.93014956, 1025.8884338 , 1025.80732602, ...,
         1026.07126183, 1026.01720529, 1025.96495023],
        [1025.76641315, 1025.73044392, 1025.67373943, ...,
         1025.92552483, 1025.85251335, 1025.79864481],
        ...,
        [1025.41083764, 1025.41164204, 1025.40809016, ...,
         1025.40879243, 1025.40590758, 1025.40698497],
        [1025.48260598, 1025.48353019, 1025.48045258, ...,
         1025.4804521 , 1025.4778254 , 1025.47890801],
        [1025.52772012, 1025.52831336, 1025.52510468, ...,
         1025.52588783, 1025.52327704, 1025.5243033 ]],

       [[1026.07555504, 1026.03119898, 1025.94031752, ...,
         1026.14606542, 1026.1246985 , 1026.09993251],
        [1025.95339821, 1025.9116772 , 1025.83058129, ...,
         1026.09450394, 1026.04044877, 1025.988199  ],
        [1025.78967357, 1025.75371197, 1025.69702506, ...,
         1025.9487692 , 1025.87576042, 1025.82189831],
...
        [1036.97836157, 1036.97839672, 1036.97842746, ...,
         1036.97835256, 1036.97834434, 1036.97834313],
        [1036.97773489, 1036.97772438, 1036.97784353, ...,
         1036.97797747, 1036.97795632, 1036.9778621 ],
        [1036.97758449, 1036.9775746 , 1036.97773408, ...,
         1036.97792061, 1036.97788401, 1036.97774809]],

       [[1036.64655067, 1036.64643097,           nan, ...,
         1036.64705517, 1036.64691441, 1036.64673658],
        [          nan,           nan,           nan, ...,
         1036.64707436, 1036.64692302,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1037.20926786, 1037.20935193, 1037.20938435, ...,
         1037.20915808, 1037.20915278, 1037.20918618],
        [1037.20870461, 1037.20875127, 1037.20880498, ...,
         1037.20868431, 1037.20867358, 1037.20868952],
        [1037.2080285 , 1037.20802132, 1037.20827329, ...,
         1037.20851279, 1037.20846717, 1037.20828487]]])
Coordinates:
  * lat       (lat) float32 244B 60.0 60.5 61.0 61.5 ... 88.5 89.0 89.5 90.0
  * pressure  (pressure) float32 324B 0.0 5.0 10.0 ... 1.85e+03 1.9e+03 1.95e+03
  * lon       (lon) float32 3kB -180.0 -179.5 -179.0 ... 178.5 179.0 179.5
Attributes:
    standard_name:  sea_water_density
    units:          kg m^-3

In [4]:
density = density.rename({"pressure" : "depth"})
density

<xarray.DataArray 'rho' (depth: 81, lat: 61, lon: 720)> Size: 28MB
array([[[1026.05231697, 1026.00796828, 1025.91707901, ...,
         1026.1228215 , 1026.10145541, 1026.0766898 ],
        [1025.93014956, 1025.8884338 , 1025.80732602, ...,
         1026.07126183, 1026.01720529, 1025.96495023],
        [1025.76641315, 1025.73044392, 1025.67373943, ...,
         1025.92552483, 1025.85251335, 1025.79864481],
        ...,
        [1025.41083764, 1025.41164204, 1025.40809016, ...,
         1025.40879243, 1025.40590758, 1025.40698497],
        [1025.48260598, 1025.48353019, 1025.48045258, ...,
         1025.4804521 , 1025.4778254 , 1025.47890801],
        [1025.52772012, 1025.52831336, 1025.52510468, ...,
         1025.52588783, 1025.52327704, 1025.5243033 ]],

       [[1026.07555504, 1026.03119898, 1025.94031752, ...,
         1026.14606542, 1026.1246985 , 1026.09993251],
        [1025.95339821, 1025.9116772 , 1025.83058129, ...,
         1026.09450394, 1026.04044877, 1025.988199  ],
        [1025.78967357, 1025.75371197, 1025.69702506, ...,
         1025.9487692 , 1025.87576042, 1025.82189831],
...
        [1036.97836157, 1036.97839672, 1036.97842746, ...,
         1036.97835256, 1036.97834434, 1036.97834313],
        [1036.97773489, 1036.97772438, 1036.97784353, ...,
         1036.97797747, 1036.97795632, 1036.9778621 ],
        [1036.97758449, 1036.9775746 , 1036.97773408, ...,
         1036.97792061, 1036.97788401, 1036.97774809]],

       [[1036.64655067, 1036.64643097,           nan, ...,
         1036.64705517, 1036.64691441, 1036.64673658],
        [          nan,           nan,           nan, ...,
         1036.64707436, 1036.64692302,           nan],
        [          nan,           nan,           nan, ...,
                   nan,           nan,           nan],
        ...,
        [1037.20926786, 1037.20935193, 1037.20938435, ...,
         1037.20915808, 1037.20915278, 1037.20918618],
        [1037.20870461, 1037.20875127, 1037.20880498, ...,
         1037.20868431, 1037.20867358, 1037.20868952],
        [1037.2080285 , 1037.20802132, 1037.20827329, ...,
         1037.20851279, 1037.20846717, 1037.20828487]]])
Coordinates:
  * lat      (lat) float32 244B 60.0 60.5 61.0 61.5 62.0 ... 88.5 89.0 89.5 90.0
  * depth    (depth) float32 324B 0.0 5.0 10.0 ... 1.85e+03 1.9e+03 1.95e+03
  * lon      (lon) float32 3kB -180.0 -179.5 -179.0 -178.5 ... 178.5 179.0 179.5
Attributes:
    standard_name:  sea_water_density
    units:          kg m^-3

In [ ]:
ds.to_netcdf(datadir + "hydrography_mimoc.nc")
density.to_netcdf(datadir + "density_mimoc.nc")